# EEE466 Lab 3: Weather Reporting with gRPC

## Part 1: Background

The objective of this lab is for you to design and implement a client-server application using remote procedure calls (RPC). RPC allows for an application run procedures on a remote machine as if it were a local machine. This is enabled through a framework of client and server stubs and an interface definition. Stubs interact with an RPC engine serialized and deserialize data. The client stub is essentially an instantiation of the server within the client program. The underlying message transport is abstracted by the RPC framework. In gRPC, the framework we will use in this lab, the server methods inherit from an automatically generated communication interface defined in protobuf.

gRPC handles server requests by using protobuf-defined classes called **services**. Services are defined in the protobuf language using the keyword `service`. **Methods** within a service are defined using the keyword `rpc`. Let's look at an example from the gRPC github repository.

```proto
// Example source: https://github.com/grpc/grpc/tree/master/examples/python/helloworld

// The greeter service definition.
service Greeter {
  // Sends a greeting
  rpc SayHello (HelloRequest) returns (HelloReply) {}
}

// The request message containing the user's name.
message HelloRequest {
  required string name = 1;
}

// The response message containing the greetings
message HelloReply {
  required string message = 1;
}
```
Every method defined as part of the service must have at least one message input and at least one message output. Note that the method `SayHello` takes `HelloRequest` as an input and returns a message object of type `HelloReply`. This example code is provided; compile the protobuf file yourself with the following commands:

In [9]:
%cd ..

C:\466labs\Lab3


In [3]:
!python -m grpc_tools.protoc helloworld.proto --proto_path=. --python_out=. --grpc_python_out=.

In [14]:
!python -m grpc_tools.protoc weather.proto --proto_path=. --python_out=. --grpc_python_out=.


Notice how this command produces a `*pb2.py` file and a `*pb2_grpc.py` file. These are python classes and methods generated by protoc, the protobuf compiler. The server inherits the communication interface in the pb2_grpc file and implements the service methods. The client imports the `GreeterStub` class from the pb2_grpc file. Both client and server import the pb2 file message types.

```py
# Server class
class Greeter(helloworld_pb2_grpc.GreeterServicer):

    # Method is inherity from super calss GreeterServicer
    def SayHello(self, request, context):
        return helloworld_pb2.HelloReply(message="Hello, %s!" % request.name)
```

The server sets up a grpc listening port 50051. Then, the client can establish an HTTP connection over TCP and engage with the service through its stub instance. 

Run the `greeter_server.py` (from https://github.com/grpc/grpc/blob/master/examples/python/helloworld/greeter_server.py) included in the example folder. With the server running, add your name to the client code below and run the code block. This will connect the client (running in this notebook) to the server, create a `HelloRequest` object, and call the remote method `SayHello` and print the result.

In [6]:
import grpc
import helloworld_pb2
import helloworld_pb2_grpc
# On the client a stub object is instantiated as if it were a class existing on the local machine
with grpc.insecure_channel("localhost:50051") as channel:
        stub = helloworld_pb2_grpc.GreeterStub(channel)
        # Make request message using imported HelloRequest Type and specifying any fields
        request = helloworld_pb2.HelloRequest(name="Andrew and Park")
        # Server methods can be called through the stub using definitions in the protobuf interface
        reply = stub.SayHello(request)
        print(reply)


message: "Hello, Andrew and Park!"



Carefully inspect how the message and service definitions in protobuf are imported into the client and server code to enable the RPC pattern in this example.

## Part 2: Design

The scenario for this lab is the design and implementation of a weather analysis server. Your server will use the `meteostat` Python library to access a database of historical daily weather records from thousands of weather stations around the globe. A client application will connect to your server to perform various analytical queries on this data. To use `meteostat` you will need to add the library to your `eee466` Anaconda environment. Run the cell below to install meteostat with the PyPI package manager, `pip`. You only need to do this once as the library will be permenantly added to your environment.

In [7]:
%pip install meteostat==1.7.4

   ---------------------------------------- 0.0/11.0 MB ? eta -:--:--
   ---------------------------------------- 11.0/11.0 MB 53.7 MB/s eta 0:00:00

   ---------------------------------------- 0/4 [pytz]
   ---------------------------------------- 0/4 [pytz]
   ---------------------------------------- 0/4 [pytz]
   ---------------------------------------- 0/4 [pytz]
   ---------- ----------------------------- 1/4 [tzdata]
   ---------- ----------------------------- 1/4 [tzdata]
   ---------- ----------------------------- 1/4 [tzdata]
   ---------- ----------------------------- 1/4 [tzdata]
   ---------- ----------------------------- 1/4 [tzdata]
   ---------- ----------------------------- 1/4 [tzdata]
   ---------- ----------------------------- 1/4 [tzdata]
   ---------- ----------------------------- 1/4 [tzdata]
   -------------------- ------------------- 2/4 [pandas]
   -------------------- ------------------- 2/4 [pandas]
   -------------------- ------------------- 2/4 [pandas]
   

In the scenario you are looking for the location with the best weather to for you and your friends to travel to for reading week. The application you build must provide two core functions:

1. Provide a summary of weather conditions over a period of time for a particular location. Includes average temperature, maximum wind speed, and total precipitation over the period.

2. Provide an analysis precipitation patterns. To improve efficiency, this function will create a **remote object** that stores precipitation data. Then this data is queried by the client repeatedly without needing to make external calls with `meteostat`.

`utils.py` contains functions to access the required information on the web. Your task will be to implement a server that provides the functionality listed above and a client that validates the correct usage of this server.

`weather.proto` contains some messages that are designed to work with `utils.py`. You must create new messages and rpc methods to complete the requirements for this lab. To compile your protobuf classes from `weather.proto`, use the following command with your `eee446` environment active:
```
python -m grpc_tools.protoc weather.proto --proto_path=. --python_out=. --grpc_python_out=.
```

## Part 3: Requirements

### Server

Your server must implement at least the following rpc methods:

1. `GetSummary`: this method fetches data for a particular lat/lon point over a time period.
  - Input: client's request must include at least `Location` and `DateRange`. `utils.fetch_summary_data()` will parse these message types and make the query with `meteostat`.
  - Output: server's response must include a message that includes the average temperature, the maximum wind, and the total precipitation.

2. `StartPrecipitationAnalysis`: this method instantiates a remote object that stores an array containing the amount of precipitation each day. 
  - Input: client's request must include at least `Location` and `DateRange`. `utils.fetch_analysis_data()` will parse these message types and make the query with `meteostat`.
  - Output: server's response must include a unique identifier for future queries. 

3. `GetPrecipitation`: this method allows the client to query precipitation data that exists on the server without the server making additional calls with `meteostat`. 
 - Input: A unique identifier associated with an existing session, a precipitation value, and an operator variable.
 - Output: server returns an error if no session with the provided identifier exists, or an array of `precipitationEvent` messages matching the query. 
 
The operator is one of: `<`, `=`, or `>`. E.g., the client specifies a particular amount of precipitation and the "greater than" operator. The server returns the `precipitationEvent` messages that exist in that session that have an associated value that is greater than the value specified in the request.

`server` does not import `meteostat` or `pandas`.

### Client

- `client` does not import `utils`.
- `client` only accesses information via RPC calls to `server`.
- `client` parses and prints the results of each request resembling the following examples.

`GetSummary()` output must include the period, average temperature, max wind, and total rainfall.
```
-- Weather Summary for Toronto (YYZ) --
Period: 2023-01-01 to 2023-12-31
Avg Temp: 10.13°C
Max Wind: 35.00 km/h
Total Rain: 858.20 mm
```

`GetPrecipitation()` output must include the query period, number of days where query is met, maximum precipitation and date, and the list of days where the query is met.
```
-- Precipitation over 10 mm in Vancouver (YVR) --
Period: 2023-01-01 to 2023-12-31
Number of days: 39
Maximum amount: 65.70 mm
Day of maximum amount: 2023-10-18
List of days:
- 2023-01-07
- 2023-01-08
- 2023-01-12
- 2023-01-13
...
```

For grading your client must make 2 summary requests to two Caribbean locations using reading week 2024 as a basline. Then, the client will make a precipitation session followed by 3 precipitation requests for this past summer in Kingston, one for each operator. Additional instructions are included in `client.py`.


## Part 4: Submission
Submit a zip archive in the format `eee446_lab3_lastname1_lastname2.zip` that includes the following:
1. A *brief* report detailing your design choices during this lab using the lab report notebook.
2. Your project folder including all python and proto files.